In [1]:
import numpy as np

import tensorflow as tf

#tfds.disable_progress_bar()

In [3]:
import argparse
import logging
import os
import pickle
import sys

import dagshub
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from common.tools import *
import common.tools as ct

In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [4]:
# parser = argparse.ArgumentParser()
# parser.add_argument("GRAPH_VER", help="version of the graph you want regex to label your CSV with", type=str)
# parser.add_argument("DATASET_PATH", help="path to your input CSV", type=str)
# args = parser.parse_args()

# GRAPH_VER = args.GRAPH_VER
# DATASET_PATH = args.DATASET_PATH

# CODE_COLUMN = "code_block"
# TARGET_COLUMN = "graph_vertex_id"

In [13]:
GRAPH_VER = "7"
DATASET_PATH = "../data/markup_data.csv"

MODEL_DIR = "../models/semi_vae_graph_v{}.sav".format(GRAPH_VER)
TFIDF_DIR = "../models/tfidf_semi_vae_graph_v{}.pickle".format(GRAPH_VER)
SUMMARY_DIR = "../models/vae_summary/"

CODE_COLUMN = "code_block"
TARGET_COLUMN = "graph_vertex_id"
RESUME = False


# ------------

BATCH_SIZE = 64

df = load_data(DATASET_PATH)
label_dim = int(np.max(df[TARGET_COLUMN].unique()) - np.min(df[TARGET_COLUMN].unique()) + 1)

kfold_params = {
    "n_splits": 15,
    "random_state": 42,
    "shuffle": True,
}

data_meta = {
    "DATASET_PATH": DATASET_PATH,
    "nrows": df.shape[0],
    "label": get_graph_vertices(GRAPH_VER),
    "model": MODEL_DIR,
    "script_dir": "nl2ml" + os.path.abspath('').split("nl2ml",1)[1] ,
}


vertices parsed: ['Hypothesis', 'Environment', 'Data_Extraction', 'EDA', 'Data_Transform', 'Model_Train', 'Model_Evaluation', 'Hyperparam_Tuning', 'Vizualization', 'Data_Export', 'Model_Deploy', 'Other']


In [11]:
df_train, df_test = train_test_split(df, test_size=0.3)

target_train = np.array(df_train[TARGET_COLUMN]).astype(int)
target_test = np.array(df_test[TARGET_COLUMN]).astype(int)
feat_train = np.array(df_train[CODE_COLUMN])
feat_test = np.array(df_test[CODE_COLUMN])


vect_text_train = ct.tfidf_fit_transform(df_train[CODE_COLUMN], {"smooth_idf": True,}, TFIDF_DIR)
vect_text_test = ct.tfidf_transform(df_test[CODE_COLUMN], {"smooth_idf": True,}, TFIDF_DIR)

In [17]:
VOCAB_SIZE = 300
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(feat_train)

In [42]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(61, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

In [43]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [44]:
history = model.fit(x=feat_train, y=target_train, epochs=10,
                    validation_data=(feat_test, target_test),
                    validation_steps=30)

Epoch 1/10
32/32 [==============================] - 24s 230ms/step - loss: 0.0000e+00 - accuracy: 0.0107 - val_loss: 0.0000e+00 - val_accuracy: 0.0138
Epoch 2/10
32/32 [==============================] - 4s 117ms/step - loss: 0.0000e+00 - accuracy: 0.0119 - val_loss: 0.0000e+00 - val_accuracy: 0.0138
Epoch 3/10
32/32 [==============================] - 4s 116ms/step - loss: 0.0000e+00 - accuracy: 0.0141 - val_loss: 0.0000e+00 - val_accuracy: 0.0138
Epoch 4/10
32/32 [==============================] - 4s 113ms/step - loss: 0.0000e+00 - accuracy: 0.0064 - val_loss: 0.0000e+00 - val_accuracy: 0.0138
Epoch 5/10
32/32 [==============================] - 4s 116ms/step - loss: 0.0000e+00 - accuracy: 0.0072 - val_loss: 0.0000e+00 - val_accuracy: 0.0138
Epoch 6/10
32/32 [==============================] - 4s 114ms/step - loss: 0.0000e+00 - accuracy: 0.0060 - val_loss: 0.0000e+00 - val_accuracy: 0.0138
Epoch 7/10
32/32 [==============================] - 4s 112ms/step - loss: 0.0000e+00 - accuracy: 0.

In [29]:
import keras

In [32]:
model = keras.Sequential()
model.add(encoder)
model.add(tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True))
model.add(keras.layers.SpatialDropout1D(0.2))
model.add(keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

In [33]:
history = model.fit(x=feat_train, y=target_train, epochs=10,
                    validation_data=(feat_test, target_test),
                    validation_steps=30)

Epoch 1/10


ValueError: in user code:

    C:\Users\nd\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\nd\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\nd\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\nd\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\nd\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\nd\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
        outputs = m